In [35]:
TECH_FOLDER_PATH = r'D:\Projects\RAG-Powered-Arabic-AI-Assistant\data\raw\Tech'
CULTURE_FOLDER_PATH = r'D:\Projects\RAG-Powered-Arabic-AI-Assistant\data\raw\Culture'
Finance_FOLDER_PATH = r'D:\Projects\RAG-Powered-Arabic-AI-Assistant\data\raw\Finance'
Medical_FOLDER_PATH = r'D:\Projects\RAG-Powered-Arabic-AI-Assistant\data\raw\Medical'
politics_FOLDER_PATH = r'D:\Projects\RAG-Powered-Arabic-AI-Assistant\data\raw\Politics'
Religion_FOLDER_PATH = r'D:\Projects\RAG-Powered-Arabic-AI-Assistant\data\raw\Religion'
Sports_FOLDER_PATH = r'D:\Projects\RAG-Powered-Arabic-AI-Assistant\data\raw\Sports'


# Specify the directory where you want to save the vector database
PERSIST_DIRECTORY = r'D:\Projects\RAG-Powered-Arabic-AI-Assistant\data\processed\vector_db'
DOC_DIRECTORY = r'D:\Projects\RAG-Powered-Arabic-AI-Assistant\data\processed\all_split.pkl'

import os
from sentence_transformers import SentenceTransformer
from langchain_chroma import Chroma
from langchain.docstore.document import Document
from tqdm import tqdm  # Import tqdm for the progress bar



In [16]:
# Initialize the model 
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

In [21]:
import pickle

# Load the all_split list from the file
with open(DOC_DIRECTORY, 'rb') as f:
    all_split2 = pickle.load(f)

In [22]:
all_split2

[Document(metadata={'source': 'D:\\Projects\\RAG-Powered-Arabic-AI-Assistant\\data\\raw\\Tech\\0000.txt'}, page_content='للعام الثاني على التوالي، حصلت شركة اس تي اس (STS) الأردنية المتخصصة في مجال تكنولوجيا المعلومات والاتصالات، على جائزتين من جوائز وورلد بيكون شركاء أي بي ام (IBM) لعام ،2008 كما فازت اس تي اس بجائزة أخرى في فئة تطبيق حلول القيادة والمطبق في جامعة العلوم والتكنولوجيا الأردنية .وأشاد رمزي الزين رئيس مجلس ادارة اس تي اس بأهمية هذه الجوائز .'),
 Document(metadata={'source': 'D:\\Projects\\RAG-Powered-Arabic-AI-Assistant\\data\\raw\\Tech\\0001.txt'}, page_content='شهدت منصة (My-HD) التلفزيونية نمواً ثابتاً في السوق الإقليمية خلال الفترة القصيرة الماضية . وقال كليف نيلسون، الرئيس التنفيذي لشركة ماي إتش دي، إن عدد المشتركين من جميع دول المنطقة قد تجاوز 000 .100 مشترك . وخلال السنة الماضية أضافت ماي إتش دي 68 قناة جديدة لمجموعة القنوات التي تقوم ببثها، وهو ما يعكس التحول المتسارع من البث التلفزيوني التقليدي (SD) إلى البث عالي الوضوح (HD).'),
 Document(metadata={'source': 'D:

In [ ]:
# all_split =  all_split[0:41600]

In [23]:
class embedding:
    def __init__(self):
        self.model = model
    def embed_documents(self , docs):
        embeddings = self.model.encode(docs)
        return embeddings.tolist()
    
    def embed_query(self , query):
        return self.model.encode(query).tolist()


In [26]:
embed_model = embedding()

# embed_model.embed_query('من هو حاكم دمينة زنجبار')

In [36]:
# Later, you can load the database like this:
loaded_vector_data = Chroma(
    persist_directory=PERSIST_DIRECTORY,
    embedding_function = embed_model
)

In [37]:
reteiever = loaded_vector_data.as_retriever(search_type = 'similarity' , search_kwargs= {'k' : 2})

In [38]:
all_split2[1].page_content

'شهدت منصة (My-HD) التلفزيونية نمواً ثابتاً في السوق الإقليمية خلال الفترة القصيرة الماضية . وقال كليف نيلسون، الرئيس التنفيذي لشركة ماي إتش دي، إن عدد المشتركين من جميع دول المنطقة قد تجاوز 000 .100 مشترك . وخلال السنة الماضية أضافت ماي إتش دي 68 قناة جديدة لمجموعة القنوات التي تقوم ببثها، وهو ما يعكس التحول المتسارع من البث التلفزيوني التقليدي (SD) إلى البث عالي الوضوح (HD).'

In [39]:
reteiever.invoke(all_split2[1].page_content)

[]

In [40]:
from transformers import AutoTokenizer, AutoModel
import torch
from tqdm import tqdm
import os
import numpy as np

# Load AraBERT
tokenizer = AutoTokenizer.from_pretrained('aubmindlab/bert-base-arabertv02')
model = AutoModel.from_pretrained('aubmindlab/bert-base-arabertv02')

# Function to generate embeddings
def embed_text(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state.mean(dim=1)
    return embeddings.squeeze().numpy()

# Embedding all documents
embeddings = []
for doc in tqdm(all_split2, desc="Embedding Documents"):
    embeddings.append(embed_text(doc.page_content))

# Convert to a NumPy array
embeddings = np.array(embeddings)


c:\Users\DELL\miniconda3\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\DELL\.cache\huggingface\hub\models--aubmindlab--bert-base-arabertv02. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
c:\Users\DELL\miniconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clea